In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
data_path=os.path.dirname(os.getcwd())+"/data_project"


In [ ]:
train_path = data_path + '/train'
img_list = []
for i in range(15):
    img_name = f'train_{str(i).zfill(2)}.png'  # zfill() pads the number with leading zeros
    img_path = os.path.join(train_path, img_name)
    if os.path.exists(img_path):
        img = cv2.imread(img_path)
        img_list.append(img)
    else:
        print(f'Image {img_name} not found.')

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(12, 8))  # create a 3x5 grid of subplots
for i in range(len(img_list)):
    row, col = divmod(i, 5)  # calculate row and column indices
    ax[row, col].imshow(img_list[i])
    ax[row, col].set_title(f'train_{str(i).zfill(2)}.png')
    ax[row, col].axis('off')  # turn off x and y axis values
plt.show()

In [ ]:
def detect_puzzle_pieces(img, min_area=1000, canny_low=100, canny_high=200):
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply Canny edge detection
    edges = cv2.Canny(blurred, canny_low, canny_high)

    # Perform morphological transformations
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=3)

    # Find contours
    contours, _ = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours by area
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

    return filtered_contours


def draw_detected_pieces(img, contours):
    # Make a copy of the original image to draw on
    img_with_contours = img.copy()

    # Draw the contours on the image
    cv2.drawContours(img_with_contours, contours, -1, (0, 0, 255), 2)

    return img_with_contours

In [ ]:
# Process each image and detect puzzle pieces
for i, img in enumerate(img_list):
    contours = detect_puzzle_pieces(img)
    img_with_contours = draw_detected_pieces(img, contours)

    # Plot the original image and the image with detected pieces
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ax[0].set_title('Original Image')
    ax[0].axis('off')

    ax[1].imshow(cv2.cvtColor(img_with_contours, cv2.COLOR_BGR2RGB))
    ax[1].set_title('Detected Pieces')
    ax[1].axis('off')

    plt.show()